In [23]:
#-*- coding : utf-8 -*-
# coding: utf-8
import numpy as np
import pandas as pd

import torch
import torch.utils.data as data

### dataset
DATA_PATH = '../Downloads/LabelMe/prepared/'
N_CLASSES = 8

In [24]:
class LabelMe_dataset(data.Dataset):
    def __init__(self, data_type, data_path):
#     def __init__(self, data_type='train', data_path):
        self.data_type = data_type
        self.data_path = data_path
        # load train data
        # images processed by VGG16
        self.data_vgg16 = np.load(self.data_path + "data_%s_vgg16.npy" % self.data_type)
        # ground truth labels
        self.labels = np.load(self.data_path + "labels_%s.npy" % self.data_type)
        
        # data from Amazon Mechanical Turk       
        if self.data_type == 'train':
            self.answers = np.load(self.data_path + "answers.npy")
#             N_ANNOT = answers.shape[1]

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        ftrs = self.data_vgg16[idx, :, :, :]
        true_label = self.labels[idx]

        if self.data_type == 'train':
            multi_label = self.answers[idx, :]
            return ftrs, true_label, multi_label
        else:
            return ftrs, true_label

In [25]:
train_data = LabelMe_dataset('train', DATA_PATH)

In [26]:
len(train_data)

10000

In [27]:
test_data = LabelMe_dataset('test', DATA_PATH)

In [28]:
len(test_data)

1188

In [45]:
### model
import torch
import torch.nn as nn
import torch.nn.functional as F

class net(nn.Module):
    def __init__(self, n_class):
        super(net, self).__init__()
        self.fc1 = nn.Linear(4*4*512, 128)
        self.fc2 = nn.Linear(128, n_class)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        out = x.view(-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = F.softmax(out)
        return out

In [46]:
model = net(n_class=8)

In [52]:
x = torch.from_numpy(train_data[0][0])

In [53]:
model(x)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([0.1228, 0.1601, 0.1384, 0.1108, 0.1481, 0.0779, 0.1211, 0.1208],
       grad_fn=<SoftmaxBackward>)

In [55]:
y_true = train_data[0][2]

In [58]:
y_true = torch.from_numpy(y_true)

In [62]:
y_true

tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  4,  4, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1,  4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1])

In [63]:
mm = torch.tensor([-1]*len(y_true))
mm

tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1])

In [65]:
y_true.eq(mm)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True])

In [68]:
torch.tensor([True, False])

tensor([ True, False])

In [69]:
torch.tensor([1,1])

tensor([1, 1])

In [70]:
torch.where(torch.tensor([True, False]), torch.tensor([1,1]), torch.tensor([2,2]))

tensor([1, 2])

In [ ]:
        >>> # input is of size N x C = 3 x 5
        >>> input = torch.randn(3, 5, requires_grad=True)
        >>> # each element in target has to have 0 <= value < C
        >>> target = torch.tensor([1, 0, 4])
        >>> output = F.nll_loss(F.log_softmax(input), target)
        >>> output.backward()

In [72]:
input = torch.randn(3, 5, requires_grad=True)
input

tensor([[ 1.8253,  1.0232, -1.3994,  0.3407,  0.7747],
        [-0.9056,  0.1924, -1.6498, -0.4747,  0.6425],
        [-0.3987, -1.4826, -0.6371, -1.1593,  1.3939]], requires_grad=True)

In [73]:
target = torch.tensor([1, 0, 4])
target

tensor([1, 0, 4])

In [74]:
output = F.nll_loss(F.log_softmax(input), target)
output

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor(1.4192, grad_fn=<NllLossBackward>)

In [75]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def one_hot(target, n_classes):
    targets = np.array([target]).reshape(-1)
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets




In [76]:
one_hot_y_true = one_hot(y_true,8)
one_hot_y_true

ValueError: only one element tensors can be converted to Python scalars

In [95]:
def one_hot(target, n_classes):
    targets = np.array([target]).reshape(-1)
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets

def multi_annot_one_hot(answers, N_CLASSES):
    row = []
    for i in range(len(answers)):
        if answers[i] == -1:
            row.append(-1 * np.ones(N_CLASSES))
        else:
            row.append(one_hot(answers[i], N_CLASSES)[0,:])
    row = np.array(row)
    return row

In [96]:
y_true

tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  4,  4, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1,  4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1])

In [97]:
multi_annot_one_hot(y_true, 8)

array([[-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1